In [1]:
!pip install transformers datasets scikit-learn rank_bm25


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [2]:
from datasets import load_dataset
dataset = load_dataset("squad")  # Use "squad_v2" for v2
print(dataset["train"][0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [3]:
# Install necessary libraries
!pip install datasets transformers scikit-learn rank_bm25

# Import libraries
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load SQuAD dataset (v1)
dataset = load_dataset("squad")

# Select a small subset (e.g., 1000 examples) for faster testing
subset = dataset["train"].select(range(1000))

# Extract contexts and questions
corpus = [item["context"] for item in subset]
questions = [item["question"] for item in subset]

# Build TF-IDF matrix on context passages
vectorizer = TfidfVectorizer().fit(corpus)
X = vectorizer.transform(corpus)

# Function to retrieve most similar context for a question
def retrieve_context(question):
    q_vec = vectorizer.transform([question])
    sims = cosine_similarity(q_vec, X).flatten()
    best_idx = sims.argmax()
    return corpus[best_idx], questions[best_idx]

# Test with a sample question
test_question = "Who was the first president of the USA?"
retrieved_context, retrieved_question = retrieve_context(test_question)

print("🔍 Your Question:")
print(test_question)
print("\n📖 Retrieved Context:")
print(retrieved_context)
print("\n🤔 Most similar training question:")
print(retrieved_question)


🔍 Your Question:
Who was the first president of the USA?

📖 Retrieved Context:
In The New Yorker music critic Jody Rosen described Beyoncé as "the most important and compelling popular musician of the twenty-first century..... the result, the logical end point, of a century-plus of pop." When The Guardian named her Artist of the Decade, Llewyn-Smith wrote, "Why Beyoncé? [...] Because she made not one but two of the decade's greatest singles, with Crazy in Love and Single Ladies (Put a Ring on It), not to mention her hits with Destiny's Child; and this was the decade when singles – particularly R&B singles – regained their status as pop's favourite medium. [...] [She] and not any superannuated rock star was arguably the greatest live performer of the past 10 years." In 2013, Beyoncé made the Time 100 list, Baz Luhrmann writing "no one has that voice, no one moves the way she moves, no one can hold an audience the way she does... When Beyoncé does an album, when Beyoncé sings a song, whe

In [4]:
from sklearn.metrics import f1_score
from tqdm import tqdm

# Get the ground-truth answers
answers = [item["answers"]["text"][0].strip() for item in subset]

# Metrics
exact_match = 0
f1_total = 0
total = len(questions)

def compute_f1(pred, truth):
    pred_tokens = pred.lower().split()
    truth_tokens = truth.lower().split()
    common = set(pred_tokens) & set(truth_tokens)
    if not common:
        return 0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

# Evaluate all QA pairs
for i in tqdm(range(total)):
    question = questions[i]
    gold_answer = answers[i]
    retrieved_context, _ = retrieve_context(question)

    if gold_answer.lower() in retrieved_context.lower():
        exact_match += 1

    f1_total += compute_f1(retrieved_context, gold_answer)

# Final metrics
em_score = exact_match / total * 100
avg_f1 = f1_total / total * 100

print(f"\n📊 Evaluation on TF-IDF (Top-1 context):")
print(f"✅ Exact Match (EM): {em_score:.2f}%")
print(f"✅ F1 Score (approx): {avg_f1:.2f}%")


100%|██████████| 1000/1000 [00:04<00:00, 226.30it/s]


📊 Evaluation on TF-IDF (Top-1 context):
✅ Exact Match (EM): 64.60%
✅ F1 Score (approx): 2.13%


In [11]:
from transformers import pipeline
from datasets import load_dataset
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import numpy as np

# ✅ Load pre-trained BERT QA pipeline
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# ✅ Load SQuAD validation dataset (100 samples for quick test)
dataset = load_dataset("squad", split="validation[:100]")
questions = dataset["question"]
answers = [a["text"][0] if a["text"] else "" for a in dataset["answers"]]
contexts_all = dataset["context"]

# ✅ TF-IDF context retrieval (top-1 matching)
vectorizer = TfidfVectorizer().fit(contexts_all)
context_vectors = vectorizer.transform(contexts_all)

def retrieve_context(question):
    q_vec = vectorizer.transform([question])
    similarity = np.dot(q_vec, context_vectors.T).toarray()[0]
    top_idx = np.argmax(similarity)
    return contexts_all[top_idx], similarity[top_idx]

# ✅ Metric functions
def compute_f1(pred, truth):
    pred_tokens = pred.lower().split()
    truth_tokens = truth.lower().split()
    common = set(pred_tokens) & set(truth_tokens)
    if not common:
        return 0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    return 2 * (precision * recall) / (precision + recall)

# ✅ Evaluate
exact_match = 0
f1_total = 0
total = len(questions)

for i in tqdm(range(total)):
    question = questions[i]
    gold_answer = answers[i]
    retrieved_context, _ = retrieve_context(question)

    try:
        result = qa_pipeline({
            "question": question,
            "context": retrieved_context
        })
        predicted_answer = result["answer"]
    except:
        predicted_answer = ""

    # EM
    if predicted_answer.strip().lower() == gold_answer.strip().lower():
        exact_match += 1

    # F1
    f1_total += compute_f1(predicted_answer, gold_answer)

# ✅ Print final score
em_score = exact_match / total * 100
avg_f1 = f1_total / total * 100

print(f"\n🎯 BERT QA Results (on TF-IDF top-1 context from SQuAD):")
print(f"✅ Exact Match (EM): {em_score:.2f}%")
print(f"✅ F1 Score: {avg_f1:.2f}%")


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
100%|██████████| 100/100 [00:03<00:00, 27.36it/s]


🎯 BERT QA Results (on TF-IDF top-1 context from SQuAD):
✅ Exact Match (EM): 56.00%
✅ F1 Score: 61.25%


In [10]:
from transformers import pipeline
from tqdm import tqdm
import numpy as np

# ✅ Replace this with your actual list of questions and answers
questions = [
    "What is the capital of France?",
    "Who wrote Harry Potter?",
    "When was the iPhone released?"
]

answers = [
    "Paris",
    "J.K. Rowling",
    "2007"
]

# ✅ Your retrieve_context() function should return a context string for a given question
def retrieve_context(question):
    # Dummy example: In actual project, use TF-IDF/BM25/etc
    context = "Paris is the capital of France. It is a popular tourist destination."
    return context, 0.9

# ✅ Load fast pretrained QA pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# ✅ Evaluation
exact_match = 0
f1_total = 0
total = len(questions)

def compute_f1(pred, truth):
    pred_tokens = pred.lower().split()
    truth_tokens = truth.lower().split()
    common = set(pred_tokens) & set(truth_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

# ✅ Run evaluation
for i in tqdm(range(total)):
    question = questions[i]
    gold_answer = answers[i]
    context, _ = retrieve_context(question)

    try:
        result = qa_pipeline({
            "question": question,
            "context": context
        })
        predicted_answer = result["answer"]
    except:
        predicted_answer = ""

    # EM
    if predicted_answer.strip().lower() == gold_answer.strip().lower():
        exact_match += 1

    # F1
    f1_total += compute_f1(predicted_answer, gold_answer)

# ✅ Final scores
em_score = exact_match / total * 100
avg_f1 = f1_total / total * 100

print(f"\n🎯 DistilBERT QA Results (on retrieved context):")
print(f"✅ Exact Match (EM): {em_score:.2f}%")
print(f"✅ F1 Score: {avg_f1:.2f}%")


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0
  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
100%|██████████| 3/3 [00:00<00:00, 55.42it/s]


🎯 DistilBERT QA Results (on retrieved context):
✅ Exact Match (EM): 33.33%
✅ F1 Score: 33.33%


In [7]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
